http://www.postgresqltutorial.com/postgresql-python/create-tables/

In [18]:
import os, time
import psycopg2
import subprocess

In [19]:
poll_freq = 10  # 10 sec
poll_timeout = 5*60     # 5 min

s3bucket = os.environ.get('AWS_S3_BUCKET')
db_host = os.environ.get('AWS_PG_DB_HOST')
db_name = os.environ.get('AWS_PG_DB_NAME')
db_user = os.environ.get('AWS_PG_DB_USER')
password = os.environ.get('AWS_PG_DB_PASS')

db_connection_string = f"dbname='{db_name}' user='{db_user}' host='{db_host}' password='{password}'"

connection = psycopg2.connect(db_connection_string)

In [20]:
def get_txn_status(connection, s3_filename, poll_freq, poll_timeout):
    cur = connection.cursor()
    sql_statement = f"select id,status from xml_txns where filename='{s3_filename}' order by status limit 1;"
    for i in range(int(poll_timeout/poll_freq)):
        cur.execute(sql_statement)
        rows = cur.fetchall()
        if len(rows):
            id_txn, status = rows[0][0], rows[0][1]
            if status == 0 or status == 1: 
                break
        time.sleep(poll_freq)

    if status == 0:
        ret_msg = "Success"
    elif status == 1:
        ret_msg = "Failed"
    else:
        ret_msg = "Timeout"

    # beyond timeout to check Lambda if fired on test-event
    cur.execute(sql_statement)
    rows = cur.fetchall()
    if len(rows) == 0:
        status, ret_msg = 1, "Lambda error"
    
    cur.close()
           
    return status, ret_msg

In [21]:
# download traffic data
# !wget -O trafficspeed.xml.gz http://opendata.ndw.nu/trafficspeed.xml.gz
cmd = "wget -O trafficspeed.xml.gz http://opendata.ndw.nu/trafficspeed.xml.gz"
wget=subprocess.run(cmd.split(), stdout=subprocess.PIPE).stdout.decode('utf-8')

In [22]:
# dt=!env TZ=Europe/Amsterdam date +%Y-%m-%d:%H%M
cmd = "env TZ=Europe/Amsterdam date +%Y-%m-%d:%H%M"
dt=subprocess.run(cmd.split(), stdout=subprocess.PIPE).stdout.decode('utf-8')
# dt[:10],dt[11:15]

In [24]:
s3_filename = f"Traffic/{dt[:10]}/{dt[11:15]}_Trafficspeed.gz"
cmd = f"aws s3 cp trafficspeed.xml.gz s3://{s3bucket}/{s3_filename}"
aws_s3_cp=subprocess.run(cmd.split(), stdout=subprocess.PIPE).stdout.decode('utf-8')

print(get_txn_status(connection, s3_filename, poll_freq, poll_timeout))

connection.close()

(0, 'Success')
